# Computing photometry and SEDs of diffsky galaxies

This notebook shows how to compute photometry through arbitrary bands of galaxies in a diffsky mock, and how to compute high-res SEDs.

## Loading the mock data

The next two cells cells show how to load the mock files that are natively produced by the diffsky source code. For purposes of this demo, we will just work with a single lightcone patch of the natively-generated hdf5 files, which will be downloaded in the next cell.

#### Using the OpenCosmo toolkit
The hdf5 files produced by diffsky are later ingested by the [OpenCosmo](https://opencosmo.readthedocs.io/en/stable/) toolkit, which enables efficient querying, map-making, and other features. See [Accessing and Working With Diffsky Mock Galaxy Catalogs](https://github.com/ArgonneCPAC/opencosmo-examples/blob/main/03-Diffsky/demo_diffmah_diffstar.ipynb) for a tutorial on how to load diffsky mocks with OpenCosmo.

In [ ]:
! wget -P ./download_dir -q -r -e robots=off -np -nH --cut-dirs=6 --reject "index.html*" https://portal.nersc.gov/project/hacc/aphearin/diffsky_data/ci_test_output/synthetic_cores/smdpl_dr1

In [ ]:
from diffsky.data_loaders.hacc_utils import load_lc_mock as llcm
import os
drn_mock = "download_dir/smdpl_dr1"
bn_mock = "lc_cores-453.0.diffsky_gals.hdf5"

diffsky_lc_patch = llcm.load_diffsky_lc_patch(drn_mock, bn_mock)

### Inspect the mock data

The `diffsky_lc_patch` dictionary stores all data columns of the mock, along with supplementary metadata needed to compute photometry.

In [ ]:
print(diffsky_lc_patch.keys())

### Recompute mock photometry

The `diffsky_lc_patch` dictionary has all the keys expected by the `compute_dbk_phot_from_diffsky_mock` function, which we use in the next two cells to show how to recompute photometry and enforce agreement with the columns stored in the mock.

In [ ]:
phot_info = llcm.compute_dbk_phot_from_diffsky_mock(**diffsky_lc_patch)

In [ ]:
import numpy as np
assert np.allclose(diffsky_lc_patch['diffsky_data']['lsst_i'], phot_info['obs_mags'][:, 3], rtol=1e-4)

### Computing photometry in other bands

You can compute photometry in other bandpasses by replacing the `tcurves` keyword argument passed to `compute_dbk_phot_from_diffsky_mock` with a different collection of transmission curves. 

Each individual transmission curve is a namedtuple with two fields: `wave` and `transmission`. In the next two cells, we'll reuse the first two transmission curves that were used to produce the original mock. But you can use any collection of transmission curves once they are formatted in the expected namedtuple.

In [ ]:
print(diffsky_lc_patch['tcurves'][0]._fields)

In [ ]:
new_tcurves = diffsky_lc_patch['tcurves'][0:2]
diffsky_lc_patch['tcurves'] = new_tcurves
phot_info = llcm.compute_dbk_phot_from_diffsky_mock(**diffsky_lc_patch)